In order not to cause lag, I put the ab testing part in a separate file to run...

In [1]:
import pandas as pd
import numpy as np
import math 
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [2]:
data = pd.read_excel("Cleaned Data.xlsx")
data.head()

,ID,Education,Income,Kids,Teens,Recency,Wines,Fruits,Meat,Fish,Sweets,Gold,Deal Purchases,Web Purchases,Catalog Purchases,Store Purchases,Web Visits Month,Cmp3 Accepted,Cmp4 Accepted,Cmp5 Accepted,Cmp1 Accepted,Cmp2 Accepted,Complain,Response,Spending,Age,Has Child,Marital Status,Generation,Income Group,Enrollment Year,Enrollment Month,Enrollment Day
0,5524,Graduation,58138.0,0,0,58,635,88,546,172,88,88,3,8,10,4,7,0,0,0,0,0,0,1,1617,66,1,Single,Baby Boomers,High,2012,9,4
1,2174,Graduation,46344.0,1,1,38,11,1,6,2,1,6,2,1,1,2,5,0,0,0,0,0,0,0,27,69,0,Single,Baby Boomers,Moderate,2014,3,8
2,4141,Graduation,71613.0,0,0,26,426,49,127,111,21,42,1,8,2,10,4,0,0,0,0,0,0,0,776,58,1,In a relationship,Generation X,Very High,2013,8,21
3,6182,Graduation,26646.0,1,0,26,11,4,20,10,3,5,2,2,0,4,6,0,0,0,0,0,0,0,53,39,0,In a relationship,Millennials,Low,2014,2,10
4,5324,PhD,58293.0,1,0,94,173,43,118,46,27,15,5,5,3,6,5,0,0,0,0,0,0,0,422,42,0,In a relationship,Millennials,High,2014,1,19


# Two sample t-test

- Campaign 4 is the campaign with the best response rate: 167
- Campaign 2 is the campaign with the lowest reponse rate: 30

In [3]:
# The campaign with the best response rate
response_cmp1 = data['Cmp1 Accepted'].sum()
response_cmp2 = data['Cmp2 Accepted'].sum()
response_cmp3 = data['Cmp3 Accepted'].sum()
response_cmp4 = data['Cmp4 Accepted'].sum()
response_cmp5 = data['Cmp5 Accepted'].sum()

print("Total responses for Campaign 1:", response_cmp1)
print("Total responses for Campaign 2:", response_cmp2)
print("Total responses for Campaign 3:", response_cmp3)
print("Total responses for Campaign 4:", response_cmp4)
print("Total responses for Campaign 5:", response_cmp5)

Total responses for Campaign 1: 142
Total responses for Campaign 2: 30
Total responses for Campaign 3: 162
Total responses for Campaign 4: 167
Total responses for Campaign 5: 158


### A/B testing compares response rates (customer acceptance) for top-performing campaigns between high-income and low-income groups.

In [4]:
high_income = data[data['Income Group'] == 'High']['Cmp4 Accepted']
low_income = data[data['Income Group'] == 'Low']['Cmp4 Accepted']

In [5]:
t_stat, p_val = stats.ttest_ind(high_income, low_income)

In [6]:
print("A/B Test 1 - T statistic:", t_stat)
print("A/B Test 1 - P value:", p_val)

A/B Test 1 - T statistic: 6.8399265336358095
A/B Test 1 - P value: 1.3531244465171712e-11


In [7]:
alpha = 0.05  # significance level: 95%

# Assuming 't_stat' and 'p_val' are the outputs of the t-test
if p_val < alpha:
    print("We reject the null hypothesis.")
else:
    print("We fail to reject the null hypothesis.")

We reject the null hypothesis.


- T-stat 6.84, large number, indicating a significant difference in response rates between the high- and low-income groups in the top-performing campaigns
- p-value < 0.05, There is a significant difference in response rates between the high-income group and the low-income group.
---------------------------------------

### A/B testing compares the average number of purchases between two groups.

In [8]:
data['Total Purchase'] = data[['Deal Purchases', 'Web Purchases', 'Catalog Purchases', 'Store Purchases']].sum(axis=1)

In [9]:
data_ab = data.copy()
data_ab = data[['Total Purchase', 'Spending', 'Income Group', 'Cmp4 Accepted']].copy()

In [10]:
# Separate data for high and low-income groups
high_income_purchase = data_ab[data_ab['Income Group'] == 'High']['Total Purchase']
low_income_purchase = data_ab[data_ab['Income Group'] == 'Low']['Total Purchase']

# Perform the independent samples t-test
t_stat, p_val = stats.ttest_ind(high_income_purchase, low_income_purchase)

# Print the results
print("A/B Test 2 - T statistic:", t_stat)
print("A/B Test 2 - P value:", p_val)

A/B Test 2 - T statistic: 32.65287682782073
A/B Test 2 - P value: 2.4826736220407915e-161


In [11]:
alpha = 0.05  # significance level: 95%

# Assuming 't_stat' and 'p_val' are the outputs of the t-test
if p_val < alpha:
    print("We reject the null hypothesis.")
else:
    print("We fail to reject the null hypothesis.")

We reject the null hypothesis.


- T-stat 32.65, large number, indicating a significant difference in average number of purchases between the two groups.
- p-value < 0.05, There is a significant difference in average number of purchases between the two groups.
---------------------------------------

### A/B test compares the average amount spent between two groups.

In [12]:
# Separate data for high and low-income groups
high_income_spend = data_ab[data_ab['Income Group'] == 'High']['Spending']
low_income_spend = data_ab[data_ab['Income Group'] == 'Low']['Spending']

# Perform the independent samples t-test
t_stat, p_val = stats.ttest_ind(high_income_spend, low_income_spend)

# Print the results
print("T-statistic:", t_stat)
print("P-value:", p_val)

T-statistic: 28.919058643140758
P-value: 2.9155065834832874e-135


In [13]:
alpha = 0.05  # significance level: 95%

# Assuming 't_stat' and 'p_val' are the outputs of the t-test
if p_val < alpha:
    print("We reject the null hypothesis.")
else:
    print("We fail to reject the null hypothesis.")

We reject the null hypothesis.


- T-stat 28.92, large number, indicating a significant difference in average spending between the two groups.
- p-value < 0.05, There is a significant difference in average spending between the two groups.
---------------------------------------

# ANOVA of Variance
- Use the ANOVA (Analysis of Variance) method to detect whether marital status affects the price (average) of their purchased products. The products examined include fish products, meat products, fruit products, desserts, wine and gold products.
- Dependent Variables are spending on various products over the past two years
- Independent Variables are marital status.

### Whether Marital status affects the price of products they buy . (mean)
- Amount spent on fish products in the past two years (MntFishProducts)
- Amount spent on meat products in the past two years (MntMeatProducts)
- Amount spent on fruit products in the past two years (MntFruits)
- Amount spent on sweet products in the past two years (MntSweetProducts)
- Amount spent on wine products in the past two years (MntWines)
- Amount spent on gold products in the past two years (MntGoldProds)

In [14]:
product_columns = ['Fish', 'Meat', 'Fruits', 'Sweets', 'Wines', 'Gold']

In [15]:
for product in product_columns:
    marital_status_categories = data['Marital Status'].unique()
    data_by_category = [data[data['Marital Status'] == category][product] for category in marital_status_categories]
    f_val, p_val = stats.f_oneway(*data_by_category)
    print(f"Product: {product}")
    print(f"F value: {f_val}")
    print(f"P value: {p_val}\n")
    
    if p_val < alpha:
        print("We reject the null hypothesis. There is a significant difference in the amount spent on this product between different marital statuses.\n")
    else:
        print("We fail to reject the null hypothesis. There is no significant difference in the amount spent on this product between different marital statuses.\n")

Product: Fish
F value: 0.27222924578919566
P value: 0.7617050635861713

We fail to reject the null hypothesis. There is no significant difference in the amount spent on this product between different marital statuses.

Product: Meat
F value: 1.2942211702333921
P value: 0.27431809688928077

We fail to reject the null hypothesis. There is no significant difference in the amount spent on this product between different marital statuses.

Product: Fruits
F value: 1.0011538157904711
P value: 0.3676211358286876

We fail to reject the null hypothesis. There is no significant difference in the amount spent on this product between different marital statuses.

Product: Sweets
F value: 0.9611743740503356
P value: 0.38260265777813884

We fail to reject the null hypothesis. There is no significant difference in the amount spent on this product between different marital statuses.

Product: Wines
F value: 2.388578226472699
P value: 0.09199594353931952

We fail to reject the null hypothesis. There is n

- None of the product types showed a significant effect of marital status on their consumption. In other words, there is no significant difference in the consumption of these products among consumers of different marital status.
--------------------------------------------

### Whether Education affects the price of products they buy . (mean)
Amount spent on fish products in the past two years (MntFishProducts)
Amount spent on meat products in the past two years (MntMeatProducts)
Amount spent on fruit products in the past two years (MntFruits)
Amount spent on sweet products in the past two years (MntSweetProducts)
Amount spent on wine products in the past two years (MntWines)
Amount spent on gold products in the past two years (MntGoldProds)

In [16]:
for product in product_columns:
    education_categories = data['Education'].unique()
    data_by_category = [data[data['Education'] == category][product] for category in education_categories]
    f_val, p_val = stats.f_oneway(*data_by_category)
    print(f"Product: {product}")
    print(f"F value: {f_val}")
    print(f"P value: {p_val}\n")
    
    if p_val < alpha:
        print("We reject the null hypothesis. There is a significant difference in the amount spent on this product between different education.\n")
    else:
        print("We fail to reject the null hypothesis. There is no significant difference in the amount spent on this product between different education.\n")

Product: Fish
F value: 12.628520543458063
P value: 3.663885481110847e-10

We reject the null hypothesis. There is a significant difference in the amount spent on this product between different education.

Product: Meat
F value: 8.40452197146084
P value: 9.981462936945208e-07

We reject the null hypothesis. There is a significant difference in the amount spent on this product between different education.

Product: Fruits
F value: 10.492844848788833
P value: 2.021143180746687e-08

We reject the null hypothesis. There is a significant difference in the amount spent on this product between different education.

Product: Sweets
F value: 12.110253836340066
P value: 9.709128480234132e-10

We reject the null hypothesis. There is a significant difference in the amount spent on this product between different education.

Product: Wines
F value: 29.66266742005684
P value: 4.5642489636633744e-24

We reject the null hypothesis. There is a significant difference in the amount spent on this product be

- All product types show a significant effect of education on their consumption. That is to say, there are significant differences in the consumption of these products among consumers with different educational levels.

==================================================================